In [1]:
import os,sys
import cv2
from scipy.spatial import distance
from os import listdir
from os.path import isfile, join
import time as t
import csv
import matplotlib.pyplot as plt
import math
import numpy as np
import easydict
from PIL import Image
from keras import backend as K

Using TensorFlow backend.


In [2]:
'''
def dice_coef(y_true, y_pred,smooth=1):
    y_true_f = K.batch_flatten(y_true)
    y_pred_f = K.batch_flatten(y_pred)
    intersection = 2 * K.sum(y_true_f * y_pred_f, axis=1, keepdims=True) + smooth
    union = K.sum(y_true_f, axis=1, keepdims=True) + \
            K.sum(y_pred_f, axis=1, keepdims=True) + \
            smooth
    return K.mean(intersection / union)
'''
def dice_fun1(im1, im2, empty_score=1.0):
    """
    Computes the Dice coefficient, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    dice : float
        Dice coefficient as a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
        Both are empty (sum eq to zero) = empty_score
        
    Notes
    -----
    The order of inputs for `dice` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)
    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / im_sum

In [3]:
proj_path = os.getcwd()
mypath = os.path.join(proj_path,'masks')
#mypath_after = os.path.join(proj_path,'img_res_kmeans')
mypath_after = os.path.join(proj_path,'img_res_meanshift')
#save_path = os.path.join(proj_path,'seg_res_kmeans')
save_path = os.path.join(proj_path,'seg_res_meanshift')
img_ori = [f for f in listdir(mypath) if isfile(join(mypath, f))]
#img = [f for f in listdir(mypath) if isfile(join(mypath_after, f))]
dice = []


In [4]:
for inputName in img_ori:
    filename, file_extension = os.path.splitext(inputName)
    filename = filename.replace("._","")
    print(filename)
    outputName = filename + '_output.png'
    #import pdb;pdb.set_trace()
    # load the two input images

    imageA = cv2.imread(os.path.join(mypath,inputName))
    if imageA is None:
        print(filename)
    else:
        imageB = cv2.imread(os.path.join(mypath_after,outputName))
        gray1 = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
        _,imageA = cv2.threshold(gray1, 60, 255, cv2.THRESH_BINARY)
        gray2 = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
        _,imageB = cv2.threshold(gray2, 60, 255, cv2.THRESH_BINARY)
        cv2.imwrite(os.path.join(save_path,outputName), imageB)
        '''
        plt.imshow(imageA)            
        plt.show()
        plt.imshow(imageB)            
        plt.show()      
        '''
        #dicevalue = dice_coef(img_ori,img)
        #import pdb;pdb.set_trace()
        dice_dist = dice_fun1(imageA, imageB)
        dice.append(dice_dist)
        print ('Dice similarity score is {}'.format(dice_dist))


crop_3601_10801_243_691_90_1
Dice similarity score is 0.8254690010074333
crop_14401_2401_332_367_0_1
Dice similarity score is 0.3263074169815073
crop_4801_8401_161_690_90_0
Dice similarity score is 0.5963784247009487
crop_4801_8401_111_145_0_0
Dice similarity score is 0.5589853054245727
crop_3601_10801_493_870_0_0
Dice similarity score is 0.8889630976381004
crop_6001_12001_952_313_90_1
Dice similarity score is 0.797244913887299
crop_14401_2401_351_166_180_0
Dice similarity score is 0.32944695360870996
crop_3601_10801_715_858_270_0
Dice similarity score is 0.8651435025144418
crop_3601_10801_512_24_270_1
Dice similarity score is 0.7799878875864942
crop_6001_12001_209_125_90_1
Dice similarity score is 0.6956930475757611
crop_6001_12001_131_561_90_0
Dice similarity score is 0.7387525158846048
crop_4801_8401_695_572_270_0
Dice similarity score is 0.661887068920404
crop_3601_10801_716_32_270_1
Dice similarity score is 0.7582853328500131
crop_6001_12001_619_906_0_1
Dice similarity score is 0.

In [5]:
#with open('kmeans_dice.csv', 'w') as fp:
#        fp.write(str(dice))
with open('ms_dice.csv', 'w') as fp:
        fp.write(str(dice))